In [1]:
#Bone Fracture Detection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

     |████████████████████████████████| 462 kB 5.3 MB/s 
  Created wheel for keras-radam: filename=keras_radam-0.15.0-py3-none-any.whl size=14686 sha256=fddc82afd1d2b1b8c1eab049278da57c52865de7774ea855f8892e8cc87ca9f1
  Stored in directory: /root/.cache/pip/wheels/23/6a/5f/d674f3b7b4d504b03148abd675e3703ba00c31763c04a2fc20
Successfully built keras-radam


In [3]:
import os
import numpy as np
import cv2
import random
import pandas as pd
from tqdm import tqdm
import keras.backend as K
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras.preprocessing.image import ImageDataGenerator, load_img#, image
from keras import regularizers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.applications.densenet import DenseNet169, DenseNet121, preprocess_input
from time import time


from focal_loss import BinaryFocalLoss
from keras_radam import RAdam

import datetime

In [4]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/MURA-v1.1.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Completed")

Completed


In [5]:
def listdir_nohidden(path):
    '''
    Utility function to find the list of files in a directory excluding the hidden files.
    Args:
        path: contains the path of the directory containing the images

    '''
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [6]:
def create_images_metadata_csv(category,study_types):
    '''
    This function creates a csv file containing the path of images, label.
    Args:
        category: train or valid, depending on which csv is needed
        study_types: the type of the body part in MURA Dataset, eg:  XR_SHOULDER
    '''
    image_data = {}
    study_label = {'positive': 1, 'negative': 0}
    #study_types = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
    #study_types = ['XR_ELBOW']
    i = 0
    image_data[category] = pd.DataFrame(columns=['Path','Count', 'Label'])
    for study_type in study_types: # Iterate throught every study types
        DATA_DIR = 'MURA-v1.1/%s/%s/' % (category, study_type)
        patients = list(os.walk(DATA_DIR))[0][1]  # list of patient folder names
        for patient in tqdm(patients):  # for each patient folder
            for study in os.listdir(DATA_DIR + patient):  # for each study in that patient folder
                if(study != '.DS_Store'):
                    label = study_label[study.split('_')[1]]  # get label 0 or 1
                    path = DATA_DIR + patient + '/' + study + '/'  # path to this study
                    for j in range(len(list(listdir_nohidden(path)))):
                        image_path = path + 'image%s.png' % (j + 1)
                        image_data[category].loc[i] = [image_path,1, label]  # add new row
                        i += 1
    image_data[category].to_csv(category+"_image_data.csv",index = None, header=False)

In [7]:
def getImageLabels(dataframe):
  '''
  This function is to get the labels of the images from the dataframe
  Args:
      dataframe: pandas dataframe containing the labels of the images

  '''
  labels = []
  for i, data in tqdm(dataframe.iterrows()):
#        img = cv2.imread(data['Path'])
#         #random rotation
#         angle = random.randint(-30,30)
#         M = cv2.getRotationMatrix2D((img_width/2,img_height/2),angle,1)
#         img = cv2.warpAffine(img,M,(img_width,img_height))
      #resize
#        img = cv2.resize(img,(img_width,img_height))    
#        img = img[...,::-1].astype(np.float32)
#        images.append(img)
      labels.append(data['Label'])
#    images = np.asarray(images).astype('float32') 
  #normalization
#    mean = np.mean(images[:, :, :])
#    std = np.std(images[:, :, :])
#    images[:, :, :] = (images[:, :, :] - mean) / std
  labels = np.asarray(labels)
  return labels

In [8]:
def getImage(dataframe, size):
    '''
    Function to process the images
    Args:
        dataframe: contains the path to the images in the directory
        size: contains the value to which the shape of the image will resized
    '''
    Images = []
    for i, data in tqdm(dataframe.iterrows()):
      try:
        image = cv2.imread(data['Path'])#cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(size,size))
        #image = randome_rotation_flip(image,size)
        Images.append(image)

      except Exception as e:
        print(str(e))

    Images = np.asarray(Images).astype('float32')

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    
    #if K.image_data_format() == "channels_first":
    #  Images = np.expand_dims(Images,axis=3)		   #Extended dimension 1
    #if K.image_data_format() == "channels_last":
    #  Images = np.expand_dims(Images,axis=3)             #Extended dimension 3(usebackend tensorflow:aixs=3; theano:axixs=1) 
    return Images

In [9]:
def build_model():
    base_model = DenseNet169(input_shape=(None, None,3),
                             weights='imagenet',
                             include_top=False,
                             pooling='avg')


    x = base_model.output

    predictions = Dense(n_classes,activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [10]:
###################
# Data processing #
###################

img_width, img_height = 224, 224
#Keras ImageDataGenerator to load, transform the images of the dataset

IMG_DATA_DIR = 'MURA-v1.1/'
train_data_dir = IMG_DATA_DIR + 'train/XR_HUMERUS'
valid_data_dir = IMG_DATA_DIR + 'valid/XR_HUMERUS'

train_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True

)
study_types = ['XR_HUMERUS']

#TRAIN DATA
create_images_metadata_csv('train',study_types)
#VALID DATA
create_images_metadata_csv('valid',study_types)

100%|██████████| 132/132 [00:00<00:00, 150.44it/s]


In [11]:
valid_image_df = pd.read_csv('valid_image_data.csv', names=['Path','Count', 'Label'])
train_image_df = pd.read_csv('train_image_data.csv', names=['Path', 'Count','Label'])

dd={}

dd['train'] = train_image_df
dd['valid'] = valid_image_df

In [12]:
train_image_labels = getImageLabels(train_image_df)

1272it [00:00, 11599.50it/s]


In [13]:
train_images = getImage(train_image_df, size = 224)

1272it [00:05, 254.08it/s]


In [14]:
train_images.shape

(1272, 224, 224, 3)

In [15]:
train_datagen.fit(train_images,augment=True)

In [16]:
train_generator = train_datagen.flow(
    x=train_images,
    y=train_image_labels,
    batch_size = 32
)

In [17]:
valid_image_labels = getImageLabels(valid_image_df)

288it [00:00, 12416.58it/s]


In [18]:
valid_images = getImage(valid_image_df, size = 224)

288it [00:01, 223.29it/s]


In [19]:
test_datagen.fit(valid_images,augment=True)

In [20]:
validation_generator = test_datagen.flow(
    x=valid_images,
    y=valid_image_labels,
    batch_size = 1
)

In [21]:
###################
# Construct model #
###################



#model parameters for training
#K.set_learning_phase(1)
nb_train_samples = len(train_images)
nb_validation_samples = len(valid_images)
epochs = 12
batch_size = 32
steps_per_epoch = nb_train_samples//batch_size
print(steps_per_epoch)
n_classes = 1

39


In [22]:
#Building a model
input_shape = (224, 224, 1)
#model = densenet(input_shape, n_classes)
model = build_model()
# Build optimizer
#opt = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.1)
#opt = RAdam(total_steps=5000, warmup_proportion=0.1, min_lr=1e-4,name='RAdam')


model.compile(loss=BinaryFocalLoss(gamma = 2), optimizer='adam', metrics=['acc', 'mse'])
#model.compile(loss=BinaryFocalLoss(gamma = 2), optimizer=opt, metrics=['acc', 'mse'])

51888128/51877672 [==============================] - 0s 0us/step


In [23]:
#callbacks for early stopping incase of reduced learning rate, loss unimprovement
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, min_lr=0.0001)
callbacks_list = [early_stop, reduce_lr]

In [24]:
#train the module
model_history = model.fit(
    train_generator,
    epochs=epochs,
    workers=0,
    use_multiprocessing=False,  
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples //batch_size,
    callbacks=callbacks_list
    
)

Epoch 1/12
39/39 [==============================] - 100s 2s/step - loss: 0.2425 - acc: 0.5661 - mse: 0.2487 - val_loss: 1.5245 - val_acc: 0.7778 - val_mse: 0.2227 - lr: 0.0010
Epoch 2/12
39/39 [==============================] - 49s 1s/step - loss: 0.1493 - acc: 0.7105 - mse: 0.2119 - val_loss: 0.2201 - val_acc: 0.5556 - val_mse: 0.2421 - lr: 0.0010
Epoch 3/12
39/39 [==============================] - ETA: 0s - loss: 0.1379 - acc: 0.7452 - mse: 0.1940
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
39/39 [==============================] - 52s 1s/step - loss: 0.1379 - acc: 0.7452 - mse: 0.1940 - val_loss: 0.5625 - val_acc: 0.7778 - val_mse: 0.1763 - lr: 0.0010
Epoch 4/12
39/39 [==============================] - 49s 1s/step - loss: 0.1332 - acc: 0.7476 - mse: 0.1940 - val_loss: 0.1396 - val_acc: 0.6667 - val_mse: 0.1441 - lr: 1.0000e-04
Epoch 5/12
39/39 [==============================] - 49s 1s/step - loss: 0.1204 - acc: 0.7976 - mse: 0.1781 - val_loss: 0.1325 